# Projecting Movie Ticket Sales & F&B Sales

**Data Sources:**

- Prop TM historical data
- All visible future TM data (sphr_visible_event_data_vf.xlsx)

In [1]:
# imports
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import statsmodels.formula.api as smf

In [2]:
tm_data = pd.read_excel('data/sphr_visible_event_data_vf.xlsx')

tm_data['Date'] = pd.to_datetime(tm_data['Date & Time'])
tm_data['Date'] = tm_data['Date'].dt.date

# Drop rows where Group Name is not Postcards
tm_data = tm_data[tm_data['Group Name'] == 'Postcards']

# Check
groups = tm_data['Group Name'].value_counts()
print(f'Movies: \n{groups}')

Movies: 
Postcards    1035
Name: Group Name, dtype: int64


In [3]:
tm_data.head()

,Group Name,Event Name,Date & Time,Average GA Ticket Price,Total Ticket Units Available,Aggregate Ticket Value,Date
30,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-05T22:00:00Z,169,5000,845000,2024-01-05
59,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-06T22:00:00Z,169,5000,845000,2024-01-06
60,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-06T00:30:00Z,169,5000,845000,2024-01-06
61,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-06T03:00:00Z,169,5000,845000,2024-01-06
62,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-07T00:30:00Z,169,5000,845000,2024-01-07


# Foreword

Movie revenue projections follows the below process:

(1) Train attendance regression model using the following independent variables:
    - Time of Day
    - Day of Week

(2) Predict attendance forward using visible Postcard movie events and their respective independent variables

(3) XXXXX Decay using Las Vegas Visitor Statistics Data
    - available on https://www.lvcva.com/research/visitor-statistics/

(4) Use historical regression model to predict ticket ASP based on attendance -> projected ticket revenues

(5) Use attendance and time of day to predict F&B sales -> projected F&B revenues

## 1. Attendance Regression Model

In [4]:
hist_postcards_attendance = pd.read_csv('prop_data/postcards_attendance.csv')
hist_postcards_attendance.head()

,Date,Day,Time,AttendancePct
0,2023-09-01,4,0,73.176026
1,2023-09-01,4,1,58.483822
2,2023-09-01,4,2,74.543956
3,2023-09-02,5,0,83.925253
4,2023-09-02,5,1,67.574545


In [5]:
formula = 'AttendancePct ~ Day + Time' # where Day is a value between 0 and 6, and Time 0 and 2 (morning, afternoon, night)

model = smf.ols(formula, data=hist_postcards_attendance).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          AttendancePct   R-squared:                       0.161
Model:                            OLS   Adj. R-squared:                  0.156
Method:                 Least Squares   F-statistic:                     29.98
Date:                Sun, 04 Feb 2024   Prob (F-statistic):           1.23e-12
Time:                        22:54:29   Log-Likelihood:                -1190.5
No. Observations:                 315   AIC:                             2387.
Df Residuals:                     312   BIC:                             2398.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     62.0720      1.565     39.670      0.0

In [6]:
# create a new df for predictions
pred_df = pd.DataFrame(columns=['Date', 'Day', 'Time', 'AttendancePct'])
pred_df['Date'] = tm_data['Date']
pred_df['Day'] = pd.to_datetime(pred_df['Date']).dt.dayofweek
# Time comes from tm_data df Date & Time column, where before 12 is 0, after 12 and before 5 is 1, and after 5 is 2
pred_df['Time'] = np.where(pd.to_datetime(tm_data['Date & Time']).dt.hour < 1, 0, np.where(pd.to_datetime(tm_data['Date & Time']).dt.hour < 5, 1, 2))
pred_df = pred_df.sort_values(by=['Date'])
pred_df = pred_df.reset_index(drop=True)

In [7]:
# get predictions and 95% CI
preds = model.get_prediction(pred_df)
preds_summary = preds.summary_frame(alpha=0.05)
preds_summary = preds_summary.reset_index(drop=True)

# Make sure no negative values or values above 100
preds_summary['mean'] = np.where(preds_summary['mean'] < 0, 0, preds_summary['mean'])
preds_summary['mean'] = np.where(preds_summary['mean'] > 100, 100, preds_summary['mean'])
preds_summary['obs_ci_lower'] = np.where(preds_summary['obs_ci_lower'] < 0, 0, preds_summary['obs_ci_lower'])
preds_summary['obs_ci_lower'] = np.where(preds_summary['obs_ci_lower'] > 100, 100, preds_summary['obs_ci_lower'])
preds_summary['obs_ci_upper'] = np.where(preds_summary['obs_ci_upper'] < 0, 0, preds_summary['obs_ci_upper'])
preds_summary['obs_ci_upper'] = np.where(preds_summary['obs_ci_upper'] > 100, 100, preds_summary['obs_ci_upper'])

# Merge in the predictions
pred_df['AttendancePct'] = preds_summary['mean']
pred_df['AttendancePct_Lower'] = preds_summary['obs_ci_lower']
pred_df['AttendancePct_Upper'] = preds_summary['obs_ci_upper']

pred_df.head()

,Date,Day,Time,AttendancePct,AttendancePct_Lower,AttendancePct_Upper
0,2024-01-05,4,2,76.969570,55.939711,97.999430
1,2024-01-06,5,2,79.384079,58.332486,100.000000
2,2024-01-06,5,0,74.144575,53.092983,95.196168
3,2024-01-06,5,1,76.764327,55.762407,97.766247
4,2024-01-06,5,2,79.384079,58.332486,100.000000


# Revenue Predicting

In [8]:
# Init Revenue Dataframe

revenue_df = pd.DataFrame()
revenue_df['Group Name'] = tm_data['Group Name']
revenue_df['Event Name'] = tm_data['Event Name']
revenue_df['Date'] = tm_data['Date']
revenue_df['Date & Time'] = tm_data['Date & Time']
revenue_df['Day'] = pd.to_datetime(revenue_df['Date']).dt.dayofweek
revenue_df['Time'] = np.where(pd.to_datetime(tm_data['Date & Time']).dt.hour < 1, 0, np.where(pd.to_datetime(tm_data['Date & Time']).dt.hour < 5, 1, 2))
revenue_df['Total Available Tickets'] = tm_data['Total Ticket Units Available']
revenue_df['Projected Attendance (Bear)'] = 0
revenue_df['Projected Attendance % (Bear)'] = 0
revenue_df['Projected Attendance (Base)'] = 0
revenue_df['Projected Attendance % (Base)'] = 0
revenue_df['Projected Attendance (Bull)'] = 0
revenue_df['Projected Attendance % (Bull)'] = 0
revenue_df['Projected Ticket Revenue (Bear)'] = 0
revenue_df['Projected Ticket Revenue (Base)'] = 0
revenue_df['Projected Ticket Revenue (Bull)'] = 0
revenue_df['Projected F&B Revenue (Bear)'] = 0
revenue_df['Projected F&B Revenue (Base)'] = 0
revenue_df['Projected F&B Revenue (Bull)'] = 0

revenue_df.head()

,Group Name,Event Name,Date,Date & Time,Day,Time,Total Available Tickets,Projected Attendance (Bear),Projected Attendance % (Bear),Projected Attendance (Base),Projected Attendance % (Base),Projected Attendance (Bull),Projected Attendance % (Bull),Projected Ticket Revenue (Bear),Projected Ticket Revenue (Base),Projected Ticket Revenue (Bull),Projected F&B Revenue (Bear),Projected F&B Revenue (Base),Projected F&B Revenue (Bull)
30,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-05,2024-01-05T22:00:00Z,4,2,5000,0,0,0,0,0,0,0,0,0,0,0,0
59,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-06,2024-01-06T22:00:00Z,5,2,5000,0,0,0,0,0,0,0,0,0,0,0,0
60,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-06,2024-01-06T00:30:00Z,5,0,5000,0,0,0,0,0,0,0,0,0,0,0,0
61,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-06,2024-01-06T03:00:00Z,5,1,5000,0,0,0,0,0,0,0,0,0,0,0,0
62,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-07,2024-01-07T00:30:00Z,6,0,5000,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
revenue_df['Projected Attendance % (Base)'] = pred_df['AttendancePct']
revenue_df['Projected Attendance % (Bear)'] = pred_df['AttendancePct_Lower']
revenue_df['Projected Attendance % (Bull)'] = pred_df['AttendancePct_Upper']

# Calculate the projected attendance
revenue_df['Projected Attendance (Base)'] = revenue_df['Total Available Tickets'] * (revenue_df['Projected Attendance % (Base)'] / 100)
revenue_df['Projected Attendance (Bear)'] = revenue_df['Total Available Tickets'] * (revenue_df['Projected Attendance % (Bear)'] / 100)
revenue_df['Projected Attendance (Bull)'] = revenue_df['Total Available Tickets'] * (revenue_df['Projected Attendance % (Bull)'] / 100)

# fill nans with the mean
revenue_df['Projected Attendance (Base)'] = revenue_df['Projected Attendance (Base)'].fillna(revenue_df['Projected Attendance (Base)'].mean())
revenue_df['Projected Attendance (Bear)'] = revenue_df['Projected Attendance (Bear)'].fillna(revenue_df['Projected Attendance (Bear)'].mean())
revenue_df['Projected Attendance (Bull)'] = revenue_df['Projected Attendance (Bull)'].fillna(revenue_df['Projected Attendance (Bull)'].mean())
revenue_df['Projected Attendance % (Bull)'] = revenue_df['Projected Attendance % (Bull)'].fillna(revenue_df['Projected Attendance % (Bull)'].mean())
revenue_df['Projected Attendance % (Base)'] = revenue_df['Projected Attendance % (Base)'].fillna(revenue_df['Projected Attendance % (Base)'].mean())
revenue_df['Projected Attendance % (Bear)'] = revenue_df['Projected Attendance % (Bear)'].fillna(revenue_df['Projected Attendance % (Bear)'].mean())

# reset index

revenue_df.head()

,Group Name,Event Name,Date,Date & Time,Day,Time,Total Available Tickets,Projected Attendance (Bear),Projected Attendance % (Bear),Projected Attendance (Base),Projected Attendance % (Base),Projected Attendance (Bull),Projected Attendance % (Bull),Projected Ticket Revenue (Bear),Projected Ticket Revenue (Base),Projected Ticket Revenue (Bull),Projected F&B Revenue (Bear),Projected F&B Revenue (Base),Projected F&B Revenue (Bull)
30,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-05,2024-01-05T22:00:00Z,4,2,5000,2906.622042,58.132441,3958.941822,79.178836,5000.000000,100.000000,0,0,0,0,0,0
59,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-06,2024-01-06T22:00:00Z,5,2,5000,3035.131227,60.702625,4089.929418,81.798588,5000.000000,100.000000,0,0,0,0,0,0
60,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-06,2024-01-06T00:30:00Z,5,0,5000,2307.096653,46.141933,3365.576709,67.311534,4424.056766,88.481135,0,0,0,0,0,0
61,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-06,2024-01-06T03:00:00Z,5,1,5000,2045.121461,40.902429,3103.601517,62.072030,4162.081574,83.241631,0,0,0,0,0,0
62,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-07,2024-01-07T00:30:00Z,6,0,5000,2178.578827,43.571577,3234.589113,64.691782,4290.599400,85.811988,0,0,0,0,0,0


In [17]:
# Attendance to ASP Regression

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Read in the CSV file
df = pd.read_csv('prop_data/postcards_asp.csv')

# Prepare the data for regression
X = df[['Theater Occupancy%']]  # Predictor variable
y = df['Average Ticket Price']  # Response variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Create a linear regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = model.predict(X_test)

print('Coefficients: \n', model.coef_)
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))
print('P-value: %.2f' % model.score(X_test, y_test))
print('Standard Error: %.2f' % np.std(y_test - y_pred))

Coefficients: 
 [0.93047519]
Mean squared error: 95.94
Coefficient of determination: 0.59
P-value: 0.59
Standard Error: 9.68


In [11]:
# apply revenues using ASP predictor
for index, row in revenue_df.iterrows():
    revenue_df.at[index, 'Projected Ticket Revenue (Base)'] = row['Projected Attendance (Base)'] * model.predict([[row['Projected Attendance % (Base)']]])[0]
    revenue_df.at[index, 'Projected Ticket Revenue (Bear)'] = row['Projected Attendance (Bear)'] * model.predict([[row['Projected Attendance % (Bear)']]])[0]
    revenue_df.at[index, 'Projected Ticket Revenue (Bull)'] = row['Projected Attendance (Bull)'] * model.predict([[row['Projected Attendance % (Bull)']]])[0]

# reset index
revenue_df = revenue_df.reset_index(drop=True)
revenue_df.head()

/Users/MilesChild/opt/miniconda3/lib/python3.9/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/MilesChild/opt/miniconda3/lib/python3.9/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/MilesChild/opt/miniconda3/lib/python3.9/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/MilesChild/opt/miniconda3/lib/python3.9/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/MilesChild/opt/miniconda3/lib/python3.9/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


,Group Name,Event Name,Date,Date & Time,Day,Time,Total Available Tickets,Projected Attendance (Bear),Projected Attendance % (Bear),Projected Attendance (Base),Projected Attendance % (Base),Projected Attendance (Bull),Projected Attendance % (Bull),Projected Ticket Revenue (Bear),Projected Ticket Revenue (Base),Projected Ticket Revenue (Bull),Projected F&B Revenue (Bear),Projected F&B Revenue (Base),Projected F&B Revenue (Bull)
0,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-05,2024-01-05T22:00:00Z,4,2,5000,2906.622042,58.132441,3958.941822,79.178836,5000.000000,100.000000,246919.329687,413843.089400,619536.699549,0,0,0
1,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-06,2024-01-06T22:00:00Z,5,2,5000,3035.131227,60.702625,4089.929418,81.798588,5000.000000,100.000000,265094.756501,437505.385714,619536.699549,0,0,0
2,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-06,2024-01-06T00:30:00Z,5,0,5000,2307.096653,46.141933,3365.576709,67.311534,4424.056766,88.481135,170249.301835,314652.927681,500755.986336,0,0,0
3,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-06,2024-01-06T03:00:00Z,5,1,5000,2045.121461,40.902429,3103.601517,62.072030,4162.081574,83.241631,140946.737255,275029.699188,450812.093928,0,0,0
4,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-07,2024-01-07T00:30:00Z,6,0,5000,2178.578827,43.571577,3234.589113,64.691782,4290.599400,85.811988,155555.085638,294522.016215,474994.010599,0,0,0


In [13]:
# export to csv
revenue_df.to_csv('prop_data/export_postcards_projection.csv', index=False)

In [12]:
# # Make a new df with date and projected attendance percentages
# attendance_df = pd.DataFrame()
# attendance_df['Date'] = pred_df['Date']
# attendance_df['Projected Attendance % (Base)'] = pred_df['AttendancePct']
# attendance_df['Projected Attendance % (Bear)'] = pred_df['AttendancePct_Lower']
# attendance_df['Projected Attendance % (Bull)'] = pred_df['AttendancePct_Upper']

# # sort by date
# attendance_df = attendance_df.sort_values(by=['Date'])
# attendance_df = attendance_df.reset_index(drop=True)
# attendance_df.head()

# # plot the attendance
# fig = go.Figure()

# fig.add_trace(go.Scatter(x=attendance_df['Date'], y=attendance_df['Projected Attendance % (Base)'], mode='lines', name='Base'))
# fig.add_trace(go.Scatter(x=attendance_df['Date'], y=attendance_df['Projected Attendance % (Bear)'], mode='lines', name='Bear'))
# fig.add_trace(go.Scatter(x=attendance_df['Date'], y=attendance_df['Projected Attendance % (Bull)'], mode='lines', name='Bull'))

# fig.update_layout(title='Projected Attendance %', xaxis_title='Date', yaxis_title='Projected Attendance %')
# fig.show()

# Food & Beverage

In [14]:
# Data taken from Sphere Menu
fnb_asps = {
    'Alcoholic Beverage ASP': 23.5,
    'Non-Alcoholic Beverage ASP': 8.75,
    'Food ASP': 21
}

# ASSUMPTION
fnb_consumption_rates = {
    'Alcohol Purchase Percent Mean': 0.2,
    'Alcohol Purchase Percent Std': 0.1,
    'Non-Alcohol Purchase Percent Mean': 0.75,
    'Non-Alcohol Purchase Percent Std': 0.15,
    'Food Purchase Percent Mean': 0.75,
    'Food Purchase Percent Std': 0.15
}

In [15]:
# apply to revenue_df
for index, row in revenue_df.iterrows():
    base_attendance = row['Projected Attendance (Base)']
    # bear case is sigma below mean
    bear_fnb = {
        'Alcohol': base_attendance * (fnb_asps['Alcoholic Beverage ASP'] * (fnb_consumption_rates['Alcohol Purchase Percent Mean'] - fnb_consumption_rates['Alcohol Purchase Percent Std'])),
        'Non-Alcohol': base_attendance * (fnb_asps['Non-Alcoholic Beverage ASP'] * (fnb_consumption_rates['Non-Alcohol Purchase Percent Mean'] - fnb_consumption_rates['Non-Alcohol Purchase Percent Std'])),
        'Food': base_attendance * (fnb_asps['Food ASP'] * (fnb_consumption_rates['Food Purchase Percent Mean'] - fnb_consumption_rates['Food Purchase Percent Std']))
    }
    # base case is mean
    base_fnb = {
        'Alcohol': base_attendance * (fnb_asps['Alcoholic Beverage ASP'] * fnb_consumption_rates['Alcohol Purchase Percent Mean']),
        'Non-Alcohol': base_attendance * (fnb_asps['Non-Alcoholic Beverage ASP'] * fnb_consumption_rates['Non-Alcohol Purchase Percent Mean']),
        'Food': base_attendance * (fnb_asps['Food ASP'] * fnb_consumption_rates['Food Purchase Percent Mean'])
    }
    # bull case is sigma above mean
    bull_fnb = {
        'Alcohol': base_attendance * (fnb_asps['Alcoholic Beverage ASP'] * (fnb_consumption_rates['Alcohol Purchase Percent Mean'] + fnb_consumption_rates['Alcohol Purchase Percent Std'])),
        'Non-Alcohol': base_attendance * (fnb_asps['Non-Alcoholic Beverage ASP'] * (fnb_consumption_rates['Non-Alcohol Purchase Percent Mean'] + fnb_consumption_rates['Non-Alcohol Purchase Percent Std'])),
        'Food': base_attendance * (fnb_asps['Food ASP'] * (fnb_consumption_rates['Food Purchase Percent Mean'] + fnb_consumption_rates['Food Purchase Percent Std']))
    }
    revenue_df.at[index, 'Projected F&B Revenue (Bear)'] = bear_fnb['Alcohol'] + bear_fnb['Non-Alcohol'] + bear_fnb['Food']
    revenue_df.at[index, 'Projected F&B Revenue (Base)'] = base_fnb['Alcohol'] + base_fnb['Non-Alcohol'] + base_fnb['Food']
    revenue_df.at[index, 'Projected F&B Revenue (Bull)'] = bull_fnb['Alcohol'] + bull_fnb['Non-Alcohol'] + bull_fnb['Food']

revenue_df.head()

,Group Name,Event Name,Date,Date & Time,Day,Time,Total Available Tickets,Projected Attendance (Bear),Projected Attendance % (Bear),Projected Attendance (Base),Projected Attendance % (Base),Projected Attendance (Bull),Projected Attendance % (Bull),Projected Ticket Revenue (Bear),Projected Ticket Revenue (Base),Projected Ticket Revenue (Bull),Projected F&B Revenue (Bear),Projected F&B Revenue (Base),Projected F&B Revenue (Bull)
0,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-05,2024-01-05T22:00:00Z,4,2,5000,2906.622042,58.132441,3958.941822,79.178836,5000.000000,100.000000,246919.329687,413843.089400,619536.699549,79970.624808,106940.915971,133911.207135
1,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-06,2024-01-06T22:00:00Z,5,2,5000,3035.131227,60.702625,4089.929418,81.798588,5000.000000,100.000000,265094.756501,437505.385714,619536.699549,82616.574249,110479.218411,138341.862572
2,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-06,2024-01-06T00:30:00Z,5,0,5000,2307.096653,46.141933,3365.576709,67.311534,4424.056766,88.481135,170249.301835,314652.927681,500755.986336,67984.649527,90912.640859,113840.632191
3,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-06,2024-01-06T03:00:00Z,5,1,5000,2045.121461,40.902429,3103.601517,62.072030,4162.081574,83.241631,140946.737255,275029.699188,450812.093928,62692.750645,83836.035980,104979.321316
4,Postcards,The Sphere Experience with Darren Aronofsky's ...,2024-01-07,2024-01-07T00:30:00Z,6,0,5000,2178.578827,43.571577,3234.589113,64.691782,4290.599400,85.811988,155555.085638,294522.016215,474994.010599,65338.700086,87374.338420,109409.976753


# Data Viz

In [16]:
# export to excel
revenue_df.to_excel('data/postcards_revenue_projection.xlsx', index=False)

In [18]:
reuploaded_df = pd.read_excel('dviz/historical_sphr_postcards_viz.xlsx')

import plotly.graph_objects as go
import pandas as pd

# Assuming 'reuploaded_df' is your DataFrame with the 'Month' and revenue columns
# Make sure 'Month' is converted to datetime if not already done
reuploaded_df['Month'] = pd.to_datetime(reuploaded_df['Month'])

fig = go.Figure()

# Bear Case in red, with shading to Base in light red
fig.add_trace(go.Scatter(x=reuploaded_df['Month'], y=reuploaded_df['Projected Ticket Revenue (Bear)'],
                         mode='lines+markers', name='Bear Case', line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=reuploaded_df['Month'].tolist() + reuploaded_df['Month'].tolist()[::-1],
                         y=reuploaded_df['Projected Ticket Revenue (Bear)'].tolist() + reuploaded_df['Projected Ticket Revenue (Base)'].tolist()[::-1],
                         fill='toself', fillcolor='rgba(255,0,0,0.2)',
                         line=dict(color='rgba(255,255,255,0)'), showlegend=False))

# Base Case in black
fig.add_trace(go.Scatter(x=reuploaded_df['Month'], y=reuploaded_df['Projected Ticket Revenue (Base)'],
                         mode='lines+markers', name='Base Case', line=dict(color='black', width=2)))

# Bull Case in lighter shade of green, with shading to Base in light green
fig.add_trace(go.Scatter(x=reuploaded_df['Month'], y=reuploaded_df['Projected Ticket Revenue (Bull)'],
                         mode='lines+markers', name='Bull Case', line=dict(color='lightgreen', width=2)))
fig.add_trace(go.Scatter(x=reuploaded_df['Month'].tolist() + reuploaded_df['Month'].tolist()[::-1],
                         y=reuploaded_df['Projected Ticket Revenue (Base)'].tolist() + reuploaded_df['Projected Ticket Revenue (Bull)'].tolist()[::-1],
                         fill='toself', fillcolor='rgba(0,255,0,0.2)',
                         line=dict(color='rgba(255,255,255,0)'), showlegend=False))

fig.update_layout(title='Projected Ticket Revenue Over Time (Including Historical Data)',
                  xaxis_title='Month', yaxis_title='Projected Revenue ($)',
                  legend=dict(y=0.5, traceorder='reversed', font_size=16))

# Note: Execute this in your local environment where Plotly can render
fig.show()